In [ ]:
import os
import cv2
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# local libraries
from utils import draw_lines

%matplotlib inline
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
figsize = (8, 6)
save_figures = False

In [ ]:
#path = '/scratch/ssd/cciw/Videos-20200114T160100Z-001/Quad1'
#file = 'GLNI_12-1_2016-07-11_video-1.mp4'
file = 'GLNI_456-3_2015-07-17_video-1.mp4'
#outpath = file.split('.')[0]

In [ ]:
#f = os.path.join(path, file)

In [ ]:
DATA_PATH = r'/scratch/gallowaa/cciw/Data'

# Search for all video files on Google Drive...
all_videos = glob(os.path.join(DATA_PATH, 'Videos_and_stills/GLNI/*/*/*/Videos/Quad*/*.mp4'))

videotable_path = os.path.join(DATA_PATH, 'Tables', 'QuadratVideos.csv')
video_df = pd.read_csv(videotable_path, index_col=0)

In [ ]:
vpath = video_df.iloc[video_df[video_df['Name'] == file].index]['Quadrat Video Path']

In [ ]:
tokens = video_df[video_df['Name'] == file]['Quadrat Video Path'].values[0].split('\\')
print(tokens)

In [ ]:
video_path = DATA_PATH + '/Videos_and_stills/GLNI'
for tok in tokens[4:-1]:
    video_path += '/' + tok
video_path = os.path.join(video_path, file)
print(video_path)  

In [ ]:
'''
out_path = os.path.join(video_path, 'data')

if not os.path.exists(out_path):
    os.mkdir(out_path)
    print('Making dir ', out_path)
'''    

## some videowriter props



In [ ]:
sz = (img.shape[1], img.shape[0])
print(sz)

In [ ]:
fps = 20
cap = cv2.VideoCapture(video_path) #cap.set(cv2.CAP_PROP_FPS, FPS)
sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
      int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print(sz)

In [ ]:
sz[0]

In [ ]:
cap.isOpened()

In [ ]:
vout = cv2.VideoWriter()
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vout.open(file.split('.')[0] + '-quadrat-demo.mp4', fourcc, fps, sz, True)

In [ ]:
currentFrame = 0

#while(True):
for i in range(20):
    
    # Capture frame-by-frame
    ret, im = cap.read()
    
    if not ret: break

    # Saves image of the current frame in jpg file
    '''
    name = 'pframe_pi45_' + str(currentFrame) + '.jpg'
    save_path = os.path.join(out_path, name)
    print ('Creating...' + name)
    '''
    
    # Do processing
    img = draw_lines(
        im, rho=1, theta=np.pi/45, mll=400, mlg=200, threshold=100, ds=1)
    
    # for saving still images
    #cv2.imwrite(save_path, img)
    
    """For annotating video
    
    @param org Bottom-left corner of the text string.
    @param org Bottom-left corner of the text string in the image.
    @param fontFace Font type, see #HersheyFonts.
    @param fontScale Font scale factor that is multiplied by the font-specific base size.
    @param color Text color.
    @param thickness Thickness of the lines used to draw a text.
    @param lineType Line type. See #LineTypes
    """
    cv2.putText(                 # x, y
        img, str(currentFrame), (50, 50), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 1, cv2.LINE_AA)
    vout.write(img)    

    # To stop duplicate images
    currentFrame += 1

# When everything done, release the capture
cap.release()
vout.release()
#cv2.destroyAllWindows()

In [ ]:
for _ in range(10):
    ret, im = cap.read()

In [ ]:
ret, im = cap.read()
print(ret)
#plt.imshow(im)

In [ ]:
y_trim = 145
x_trim = 1
img, edges = draw_lines(
    im[y_trim:-y_trim, x_trim:-x_trim, :], rho=1, theta=np.pi/45, mll=400, mlg=200, threshold=50, ds=1)

In [ ]:
#plt.figure(figsize=(14, 12))
plt.imshow(im[y_trim:-y_trim, x_trim:-x_trim, :])

In [ ]:
plt.imshow(edges)

In [ ]:
#cv2.circle(img, (800, 400), 10, (255, 0, 0), thickness=2, lineType=8, shift=0)
plt.figure(figsize=figsize)
plt.imshow(img)
plt.tight_layout()
if save_figures:
    plt.savefig('img/' + outpath + '-Step-2.png')
plt.show()

In [ ]:
plt.imshow(edges)